Notebook to reshape the regular ems data as provided by Alliander to the spatial dataset, where each measurement of each field within a station is a column. Takes a long time to run.

In [1]:
import pyarrow.feather as feather
import pandas as pd
from datetime import datetime
import numpy as np
import math

import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt

In [2]:
# Calculates current from P, Q, and U
def calculate_I(P,Q,U):
    return np.sqrt(pow(P,2) + pow(Q,2)/(U*np.sqrt(3)))

# Calculate phase angle in radians
def calculate_theta(P,Q):
    if P == 0:
        theta = math.pi*0.5
    else:
        theta = math.atan(Q/P)
        
    if P < 0 & Q > 0:
        print(P,Q)
        theta += 2*math.pi
    elif Q < 0:
        theta += math.pi
    
    return theta

# converts an amount in 'nanoseconds' to an amount in '5 minutes'
def ns_to_5m(x):
    return x/(pow(10,9)*60*5)

# Interweaves list of lists: [lst1[0], lst2[0], lst[3], lst1[1], lst2[1], lst3[1], etc]
# Assumes equal length lists. Can be done with N lists.
def interweave_lists(lists):
    result = [val for tup in zip(*lists) for val in tup]
    return result


def adjust_col_names(col_names, name1, name2):
    for index,old_name in enumerate(col_names[2:]):
        if index%2 == 0:
            col_names[index+2] = name1 + str(int((old_name-2)/2))
        else:
            col_names[index+2] = name2 + str(int((old_name-3)/2))
    return col_names


# Gets values for P, Q, and I and checks if only I is NaN
def only_I_is_nan(P, Q, I):
    value = (not np.isnan(P) and not np.isnan(Q)) and np.isnan(I)
    return value


def fill_nan(temp_df, mean_P, mean_Q, mean_I):
    # Calculate I from P and Q only if I is NaN and P and Q are not NaNs
    temp_df["M_VALUE_I"] = temp_df.apply(lambda x: calculate_I(x["M_VALUE_P"], x["M_VALUE_Q"], x["BEDRIJFSSPANNING"]) if only_I_is_nan(x["M_VALUE_P"], x["M_VALUE_Q"], x["M_VALUE_I"]) else x["M_VALUE_I"], axis=1)

    temp_df["M_VALUE_P"] = temp_df["M_VALUE_P"].fillna(value= mean_P)
    temp_df["M_VALUE_Q"] = temp_df["M_VALUE_Q"].fillna(value= mean_Q)
    temp_df["M_VALUE_I"] = temp_df["M_VALUE_I"].fillna(value= mean_I)
    return temp_df

In [3]:
def open_file(file_name):
    df = feather.read_feather(file_name)
    df['DATUM_TIJD'] = pd.to_datetime(df['DATUM_TIJD'])
    return df

In [4]:
def pre_processing(df):
    # preprocessing

    # Dropping columns
    remove_list = [0,1,5,6,7,8,10,11,12,14,15,16]
    df = df.drop(df.columns[remove_list], axis=1)
    stations = df["TA_B1_NAME"].unique()
    
    # sort values on "DATUM_TIJD"
    df = df.sort_values("DATUM_TIJD")
    start_time = ns_to_5m(df["DATUM_TIJD"].iloc[1].value)
    
    # Fill nans
    mean_P = df["M_VALUE_P"].mean()
    mean_Q = df["M_VALUE_Q"].mean()
    mean_I = df["M_VALUE_I"].mean()

    df = fill_nan(df, mean_P, mean_Q, mean_I)
    return df

In [5]:
def reshape_data(df):
    dfs_IU = []
    dfs_PQ = []

    for index, station in enumerate(df["TA_B1_NAME"].unique()):
        station_df = df[df["TA_B1_NAME"] == station]
        print(station)
        IU = []
        PQ = []

        for timestamp in station_df["DATUM_TIJD"].unique():
            timestamp_df = station_df[station_df["DATUM_TIJD"] == timestamp]

            lst_I = timestamp_df["M_VALUE_I"].tolist()
            lst_U = timestamp_df["BEDRIJFSSPANNING"].tolist()

            lst_P = timestamp_df["M_VALUE_P"].tolist()
            lst_Q = timestamp_df["M_VALUE_Q"].tolist()

            #Interlacing two lists and add timestamp and station
            lst_IU = interweave_lists([lst_I, lst_U])
            lst_IU.insert(0,timestamp)
            lst_IU.insert(1,station)

            lst_PQ = interweave_lists([lst_P, lst_Q])
            lst_PQ.insert(0,timestamp)
            lst_PQ.insert(1,station)

            IU.append(lst_IU)
            PQ.append(lst_PQ)

        df_IU = pd.DataFrame(IU)
        df_PQ = pd.DataFrame(PQ)

        df_IU.rename(columns={0: "DATUM_TIJD", 1: "STATION"}, inplace=True)
        df_PQ.rename(columns={0: "DATUM_TIJD", 1: "STATION"}, inplace=True)

        dfs_IU.append(df_IU)
        dfs_PQ.append(df_PQ)
        
    return dfs_IU, dfs_PQ

In [6]:
def combine_dfs(dfs, name1, name2):
    for index, df_temp in enumerate(dfs):
        dfs[index] = df_temp

    df_new = pd.concat(dfs, ignore_index = True)


    col_names = df_new.columns.tolist()
    col_names = adjust_col_names(col_names, name1, name2)

    df_new.columns = col_names
    return df_new

In [7]:
def run():
    file_name = "ems_metingen_pqi.feather"
    df = open_file(file_name)
    df = pre_processing(df)

    df_IU, df_PQ = reshape_data(df)

    df_IU_new = combine_dfs(df_IU,"I_","U_")
    df_PQ_new = combine_dfs(df_PQ,"P_","Q_")

    df_IU_new.to_feather("df_IU.feather")
    df_PQ_new.to_feather("df_PQ.feather")
    
    return df_IU_new, df_PQ_new
    
df_IU, df_PQ = run()

Tex
Nk
Dvd-RS
HrvH
Grd
HFDP
Dtn
Hby
Ns
Lw


In [26]:
df_PQ

,DATUM_TIJD,STATION,P_0,Q_0,P_1,Q_1,P_2,Q_2,P_3,Q_3,...,P_20,Q_20,P_21,Q_21,P_22,Q_22,P_23,Q_23,P_24,Q_24
0,2021-01-01 00:00:00,Tex,0.00,0.00,1.29,-0.53,0.75,0.10,0.87,-0.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 00:05:00,Tex,-0.07,0.07,0.00,0.00,0.71,-0.28,0.75,-0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 00:10:00,Tex,0.67,-0.28,0.00,0.00,-8.07,1.88,-0.07,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 00:15:00,Tex,1.24,-0.52,1.06,-0.20,0.40,0.01,0.86,-0.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 00:20:00,Tex,1.24,-0.52,-0.07,0.07,-3.79,0.84,0.40,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051014,2021-12-31 23:35:00,Lw,3.68,-8.41,16.93,-3.69,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051015,2021-12-31 23:40:00,Lw,3.87,-8.54,16.73,-3.70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051016,2021-12-31 23:45:00,Lw,3.15,-8.55,16.84,-3.70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051017,2021-12-31 23:50:00,Lw,3.23,-8.45,16.60,-3.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
